## 准备数据

In [ ]:
import os
import sys

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets
import warnings

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}
warnings.filterwarnings('ignore')  # or any {'ignore', 'always', 'default', 'error', 'fatal', 'warn'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    # x = x/255.0
    # x_test = x_test/255.0
    x = x.reshape(-1, 28*28).astype('float32') / 255.0  # Flatten images
    x_test = x_test.reshape(-1, 28*28).astype('float32') / 255.0
    return (x, y), (x_test, y_test)

In [ ]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

## 建立模型

In [ ]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(tf.random.normal([28 * 28, 128], stddev=0.01))  # 输入层到隐藏层
        self.b1 = tf.Variable(tf.zeros([128]))  # 隐藏层偏置
        self.W2 = tf.Variable(tf.random.normal([128, 10], stddev=0.01))  # 隐藏层到输出层
        self.b2 = tf.Variable(tf.zeros([10]))  # 输出层偏置
        
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        # 输入层到隐藏层
        z1 = tf.matmul(x, self.W1) + self.b1
        a1 = tf.nn.relu(z1)  # 使用 ReLU 作为激活函数
        logits = tf.matmul(a1, self.W2) + self.b2  # 输出未归一化的 logits
        
        return logits
        
model = myModel()

optimizer = optimizers.Adam(learning_rate=0.01)

## 计算 loss

In [ ]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y, alpha):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(alpha*g)
    # optimizer.apply_gradients(zip(grads, trainable_vars))
    
    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [ ]:
train_data, test_data = mnist_dataset()
alpha = 1
for epoch in range(50):
    if epoch % 10 == 0:
        alpha *= 0.8
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64),
                                    alpha=alpha)
    print('\repoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy(), end='')
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))
print('\ntest loss', loss.numpy(), '; accuracy', accuracy.numpy(), end='')